# Affordable Housing Data - ETL

This notebook contains the code for the ETL of the Affordable Housing Data.

In [92]:
import numpy as np
import pandas as pd

## Extract

In [93]:
census_housing = pd.read_csv("./data/JAX-CensusHousing.csv")
census_incomes = pd.read_csv("./data/JAX-CensusHouseholdIncome.csv")

print("Census Housing Data Samples: ", census_housing["geoid"].unique().shape[0])
print(
    "Census Household Income Data Samples: ", census_incomes["geoid"].unique().shape[0]
)

Census Housing Data Samples:  208
Census Household Income Data Samples:  208


In [94]:
# View the first few rows of Census Housing dataset
census_housing.head(2)

,feature id,feature label,shid,geoid,indicator name,indicator time,indicator unit,indicator format,indicator source,value
0,publicpolygonfeature:8:5404039,Tract 12031014421,country:us/state:fl/tract:12031014421,12031014421,Total Housing Units,2019-2023,Housing Units,number,US Census Bureau ACS 5-year,2394.0
1,publicpolygonfeature:8:5404039,Tract 12031014421,country:us/state:fl/tract:12031014421,12031014421,Housing Units with SNAP Benefits (USDA),2019,Housing Units,number,USDA ERS FARA,117.0


In [95]:
# View the first few rows of Census Household Income dataset
census_incomes.head(2)

,feature id,feature label,shid,geoid,indicator name,indicator time,indicator unit,indicator format,indicator source,value
0,publicpolygonfeature:8:5404039,Tract 12031014421,country:us/state:fl/tract:12031014421,12031014421,Households Receiving SNAP,2019-2023,Households,number,US Census Bureau ACS 5-year,16.0
1,publicpolygonfeature:8:5404039,Tract 12031014421,country:us/state:fl/tract:12031014421,12031014421,Households Not Receiving SNAP,2019-2023,Households,number,US Census Bureau ACS 5-year,2330.0


## Transform: Data Restructuring

In [96]:
FILLER_COLUMNS: list[str] = [
    "feature id",
    "feature label",
    "shid",
    "indicator time",
    "indicator unit",
    "indicator format",
    "indicator source",
]

# Remove filler columns
census_housing_cleaned: pd.DataFrame = census_housing.drop(columns=FILLER_COLUMNS)
census_incomes_cleaned: pd.DataFrame = census_incomes.drop(columns=FILLER_COLUMNS)

census_housing_cleaned.head(2)

,geoid,indicator name,value
0,12031014421,Total Housing Units,2394.0
1,12031014421,Housing Units with SNAP Benefits (USDA),117.0


In [97]:
# Pivot the table on geoid to have features as columns
census_housing_pivoted: pd.DataFrame = census_housing_cleaned.pivot_table(
    index="geoid", columns="indicator name", values="value"
).reset_index()
census_incomes_pivoted: pd.DataFrame = census_incomes_cleaned.pivot_table(
    index="geoid", columns="indicator name", values="value"
).reset_index()

print("Census Housing Pivoted Shape: ", census_housing_pivoted.shape)
print("Census Incomes Pivoted Shape: ", census_incomes_pivoted.shape)
census_housing_pivoted.head(2)

Census Housing Pivoted Shape:  (208, 20)
Census Incomes Pivoted Shape:  (208, 26)


indicator name,geoid,HUD Subsidized Housing Units,Housing Insecurity Among Adults,Housing Units (USDA),Housing Units with No Vehicles (USDA),Housing Units with SNAP Benefits (USDA),LIHTC Qualified Census Tract,Median Home Costs as a Percentage of Income with a Mortgage,Median Home Costs as a Percentage of Income without a Mortgage,Median Home Rent,Median Home Value,Overcrowded Housing Units,Percent HUD Assisted Housing with Minority Householder,Potential for Lead Paint (Housing Built 1979 and Earlier),Total Housing Units,Total Occupied Housing Units,Total Owner Occupied Housing Units,Total Renter Occupied Housing Units,Total Vacant Housing Units,Walkability Employment-Housing Mix Ranking
0,1235000,17003.0,16.5,323233.0,27709.0,56156.0,59.0,21.7,10.3,1375.0,266100.0,8893.0,83.0,171673.0,422355.0,384741.0,220976.0,163765.0,37614.0,11.0
1,12031000101,61.0,20.1,1090.0,212.0,381.0,1.0,10.7,15.9,929.0,126600.0,2.0,97.0,1342.0,1840.0,1461.0,910.0,551.0,379.0,NaN


In [98]:
# Combine the datasets on geoid
combined_data: pd.DataFrame = pd.merge(
    census_housing_pivoted, census_incomes_pivoted, on="geoid", how="inner"
)
print("Combined Data Shape: ", combined_data.shape)
combined_data.head(2)

Combined Data Shape:  (208, 45)


indicator name,geoid,HUD Subsidized Housing Units,Housing Insecurity Among Adults,Housing Units (USDA),Housing Units with No Vehicles (USDA),Housing Units with SNAP Benefits (USDA),LIHTC Qualified Census Tract,Median Home Costs as a Percentage of Income with a Mortgage,Median Home Costs as a Percentage of Income without a Mortgage,Median Home Rent,...,Median Household Income,Median Monthly Ownership Costs as a Percentage of Income,Owner Occupied Household Median Income,"Percent HUD Assisted Housing with Household Income $10,000 to $14,999","Percent HUD Assisted Housing with Household Income $15,000 to $19,999","Percent HUD Assisted Housing with Household Income $20,000 or More","Percent HUD Assisted Housing with Household Income $4,999 or Less","Percent HUD Assisted Housing with Household Income $5,000 to $9,999",Renter Excessive Housing Costs,Renter Occupied Household Median Income
0,1235000,17003.0,16.5,323233.0,27709.0,56156.0,59.0,21.7,10.3,1375.0,...,66981.0,18.1,84779.0,37.0,14.0,27.0,15.0,7.0,84587.0,48932.0
1,12031000101,61.0,20.1,1090.0,212.0,381.0,1.0,10.7,15.9,929.0,...,39369.0,14.1,NaN,39.0,20.0,24.0,7.0,9.0,345.0,37124.0


In [99]:
# Keep only relevant columns for analysis
RELEVANT_COMMON_COLUMNS = [
    "Total Housing Units",
    "Overcrowded Housing Units",
    "Total Occupied Housing Units",
    "Total Vacant Housing Units",
    "Households Receiving SNAP",
    "Households Not Receiving SNAP",
    "Households with Public Assistance Income",
    "Households without Public Assistance Income",
    "Median Household Income",
    "HUD Subsidized Housing Units",
]
RELEVANT_RENTER_COLUMNS = [
    "Total Renter Occupied Housing Units",
    "Renter Occupied Household Median Income",
    "Median Gross Rent as a Percentage of Income",
    "Renter Excessive Housing Costs",
]
RELEVANT_OWNER_COLUMNS = [
    "Total Owner Occupied Housing Units",
    "Owner Occupied Household Median Income",
    "Median Monthly Ownership Costs as a Percentage of Income",
    "Home Owner Excessive Housing Costs",
]

renter_data = combined_data[
    ["geoid"] + RELEVANT_COMMON_COLUMNS + RELEVANT_RENTER_COLUMNS
]
owner_data = combined_data[["geoid"] + RELEVANT_COMMON_COLUMNS + RELEVANT_OWNER_COLUMNS]

print("Renter Data Shape: ", renter_data.shape)
print("Owner Data Shape: ", owner_data.shape)
renter_data.head(2)

Renter Data Shape:  (208, 15)
Owner Data Shape:  (208, 15)


indicator name,geoid,Total Housing Units,Overcrowded Housing Units,Total Occupied Housing Units,Total Vacant Housing Units,Households Receiving SNAP,Households Not Receiving SNAP,Households with Public Assistance Income,Households without Public Assistance Income,Median Household Income,HUD Subsidized Housing Units,Total Renter Occupied Housing Units,Renter Occupied Household Median Income,Median Gross Rent as a Percentage of Income,Renter Excessive Housing Costs
0,1235000,422355.0,8893.0,384741.0,37614.0,56051.0,328690.0,11895.0,372846.0,66981.0,17003.0,163765.0,48932.0,32.2,84587.0
1,12031000101,1840.0,2.0,1461.0,379.0,252.0,1209.0,49.0,1412.0,39369.0,61.0,551.0,37124.0,33.9,345.0


## Transform: Feature Engineering

In [100]:
# Feature Labels
FE_COMMON = {
    "Occupancy Rate": [
        "Total Occupied Housing Units",
        "Total Housing Units",
    ],  # Total Occupied Housing Units / Total Housing Units
    "Vacancy Rate": [
        "Total Vacant Housing Units",
        "Total Housing Units",
    ],  # Total Vacant Housing Units / Total Housing Units
    "Overcrowding Rate": [
        "Overcrowded Housing Units",
        "Total Occupied Housing Units",
    ],  # Overcrowded Housing Units / Total Occupied Housing Units
    "SNAP Percentage": [
        "Households Receiving SNAP",
        "Total Housing Units",
    ],  # Households Receiving SNAP / Total Housing Units
    "Public Assistance Percentage": [
        "Households with Public Assistance Income",
        "Total Housing Units",
    ],  # Households with Public Assistance Income / Total Housing Units
    "HUD Subsidized Housing Percentage": [
        "HUD Subsidized Housing Units",
        "Total Housing Units",
    ],  # HUD Subsidized Housing Units / Total Housing Units
}
FE_RENTER: dict[str, list[str | int]] = {
    "Rent Rate": [
        "Total Renter Occupied Housing Units",
        "Total Occupied Housing Units",
    ],
    "Rent to Overall Median Income Ratio": [
        "Renter Occupied Household Median Income",
        "Median Household Income",
    ],
    "Excessive Rent Rate": [
        "Renter Excessive Housing Costs",
        "Total Renter Occupied Housing Units",
    ],
    "Rent Cost to Income Ratio": [
        "Median Gross Rent as a Percentage of Income",
        100,
    ],
}
FE_OWNER: dict[str, list[str | int]] = {
    "Ownership Rate": [
        "Total Owner Occupied Housing Units",
        "Total Occupied Housing Units",
    ],
    "Ownership to Overall Median Income Ratio": [
        "Owner Occupied Household Median Income",
        "Median Household Income",
    ],
    "Excessive Ownership Rate": [
        "Home Owner Excessive Housing Costs",
        "Total Owner Occupied Housing Units",
    ],
    "Ownership Cost to Income Ratio": [
        "Median Monthly Ownership Costs as a Percentage of Income",
        100,
    ],
}


def engineer_features(
    source: pd.DataFrame, features: dict[str, list[str | int]]
) -> pd.DataFrame:
    destination = source.copy()
    for feature, columns in features.items():
        if isinstance(columns[1], int):
            destination[feature] = destination[columns[0]] / columns[1]
        else:
            destination[feature] = destination[columns[0]] / destination[columns[1]]
    return destination


renter_data_fe = engineer_features(engineer_features(renter_data, FE_COMMON), FE_RENTER)
renter_data_fe = renter_data_fe[
    ["geoid"] + list(FE_COMMON.keys()) + list(FE_RENTER.keys())
]
owner_data_fe = engineer_features(engineer_features(owner_data, FE_COMMON), FE_OWNER)
owner_data_fe = owner_data_fe[
    ["geoid"] + list(FE_COMMON.keys()) + list(FE_OWNER.keys())
]


print("Renter Data with Features Shape: ", renter_data_fe.shape)
print("Owner Data with Features Shape: ", owner_data_fe.shape)
renter_data_fe.head(2)

Renter Data with Features Shape:  (208, 11)
Owner Data with Features Shape:  (208, 11)


indicator name,geoid,Occupancy Rate,Vacancy Rate,Overcrowding Rate,SNAP Percentage,Public Assistance Percentage,HUD Subsidized Housing Percentage,Rent Rate,Rent to Overall Median Income Ratio,Excessive Rent Rate,Rent Cost to Income Ratio
0,1235000,0.910942,0.089058,0.023114,0.132711,0.028164,0.040258,0.425650,0.730536,0.516515,0.322
1,12031000101,0.794022,0.205978,0.001369,0.136957,0.026630,0.033152,0.377139,0.942975,0.626134,0.339


## Load

In [101]:
# Remove rows with NaN or infinite values
renter_data_fe.replace([np.inf, -np.inf], np.nan, inplace=True)
renter_data_fe.dropna(inplace=True)
owner_data_fe.replace([np.inf, -np.inf], np.nan, inplace=True)
owner_data_fe.dropna(inplace=True)

# Save the processed data to CSV files
print("Final Renter Data Shape: ", renter_data_fe.shape)
print("Final Owner Data Shape: ", owner_data_fe.shape)
renter_data_fe.to_csv("./data/JAX-Renter-Features.csv", index=False)
owner_data_fe.to_csv("./data/JAX-Owner-Features.csv", index=False)

Final Renter Data Shape:  (174, 11)
Final Owner Data Shape:  (199, 11)


In [102]:
# Test Reloading the saved files
renter_data_loaded = pd.read_csv("./data/JAX-Renter-Features.csv")
renter_data_loaded.head(2)

,geoid,Occupancy Rate,Vacancy Rate,Overcrowding Rate,SNAP Percentage,Public Assistance Percentage,HUD Subsidized Housing Percentage,Rent Rate,Rent to Overall Median Income Ratio,Excessive Rent Rate,Rent Cost to Income Ratio
0,1235000,0.910942,0.089058,0.023114,0.132711,0.028164,0.040258,0.425650,0.730536,0.516515,0.322
1,12031000101,0.794022,0.205978,0.001369,0.136957,0.026630,0.033152,0.377139,0.942975,0.626134,0.339
